# Artificial Intelligence Nanodegree
## Machine Translation Project
In this notebook, sections that end with **'(IMPLEMENTATION)'** in the header indicate that the following blocks of code will require additional functionality which you must provide. Please be sure to read the instructions carefully!

## Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [1]:
import helper


# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [2]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les Ã©tats-unis est gÃ©nÃ©ralement froid en juillet , et il gÃ¨le habituellement en novembre .


From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save you some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [3]:
import collections


english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `english_sentences` and `french_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [4]:
import project_tests as tests
from keras.preprocessing.text import Tokenizer

def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement
    t = Tokenizer()
    t.fit_on_texts(x)
    sequences = t.texts_to_sequences(x)
    
    return sequences, t

tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

Using TensorFlow backend.


{'this': 18, 'over': 7, 'sentence': 21, 'brown': 4, 'lexicography': 15, 'jove': 11, 'short': 20, 'the': 1, 'of': 14, 'my': 12, 'fox': 5, 'lazy': 8, 'study': 13, 'won': 16, 'jumps': 6, 'a': 3, 'by': 10, 'prize': 17, 'quick': 2, 'dog': 9, 'is': 19}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [5]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences


def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    return pad_sequences(x, maxlen=length, padding='post')

tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [6]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)

print('Data Preprocessed')

Data Preprocessed


## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an optional Encoder-Decoder RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.  You'll be using this function to better understand the output of the neural network.

In [7]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: RNN (IMPLEMENTATION)
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

In [8]:
from keras.layers import GRU, Input, Dense, TimeDistributed
from keras.models import Model
from keras.layers import Activation
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy


def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Build the layers
    rnn_size = english_vocab_size    #--- I found that setting the rnn output size to the vocab size of the input works 
    sentence_shape = (input_shape[1], input_shape[2])
    
    rnn_input  = Input(shape=sentence_shape)
    rnn        = GRU(rnn_size, return_sequences=True)      (rnn_input)
    rnn        = TimeDistributed(Dense(french_vocab_size)) (rnn)
    rnn_output = Activation("softmax")                     (rnn)
    
    model = Model(inputs=rnn_input, outputs=rnn_output)
    
    learning_rate = 0.0075
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model


tests.test_simple_model(simple_model)


# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index),
    len(french_tokenizer.word_index))

simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2, verbose=2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
10s - loss: 1.8015 - acc: 0.5674 - val_loss: nan - val_acc: 0.6332
Epoch 2/10
8s - loss: 1.1698 - acc: 0.6525 - val_loss: nan - val_acc: 0.6728
Epoch 3/10
8s - loss: 1.0425 - acc: 0.6700 - val_loss: nan - val_acc: 0.6806
Epoch 4/10
8s - loss: 0.9685 - acc: 0.6833 - val_loss: nan - val_acc: 0.6854
Epoch 5/10
8s - loss: 0.9175 - acc: 0.6962 - val_loss: nan - val_acc: 0.7027
Epoch 6/10
8s - loss: 0.8735 - acc: 0.7088 - val_loss: nan - val_acc: 0.7242
Epoch 7/10
8s - loss: 0.8239 - acc: 0.7255 - val_loss: nan - val_acc: 0.7383
Epoch 8/10
8s - loss: 0.8133 - acc: 0.7253 - val_loss: nan - val_acc: 0.7160
Epoch 9/10
8s - loss: 0.8010 - acc: 0.7208 - val_loss: nan - val_acc: 0.7235
Epoch 10/10
8s - loss: 0.7779 - acc: 0.7288 - val_loss: nan - val_acc: 0.7290
new jersey est parfois chaud en mois de il automne il est neigeux en <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


### Model 2: Embedding (IMPLEMENTATION)
![RNN](images/embedding.png)
You've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

In [9]:
from keras.layers.embeddings import Embedding


def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    rnn_size = english_vocab_size    #--- I found that setting the rnn output size to the vocab size of the input works 
    
    embedding_input = Input(shape=(input_shape[1],))
    embedding       = Embedding(english_vocab_size, 64, input_length=input_shape[1])(embedding_input)

    rnn             = GRU(rnn_size, return_sequences=True)      (embedding)
    rnn             = TimeDistributed(Dense(french_vocab_size)) (rnn)
    rnn_output      = Activation("softmax")                     (rnn)
    
    model = Model(inputs=embedding_input, outputs=rnn_output)
    
    learning_rate = 0.0075
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model

tests.test_embed_model(embed_model)

# TODO: Reshape the input for embedding
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[1]))

# TODO: Train the neural network
simple_embed_model = embed_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index),
    len(french_tokenizer.word_index))

simple_embed_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2, verbose=2)

# TODO: Print prediction(s)
print(logits_to_text(simple_embed_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
10s - loss: 2.0121 - acc: 0.5784 - val_loss: nan - val_acc: 0.7875
Epoch 2/10
9s - loss: 0.5200 - acc: 0.8448 - val_loss: nan - val_acc: 0.8835
Epoch 3/10
9s - loss: 0.3208 - acc: 0.8975 - val_loss: nan - val_acc: 0.9081
Epoch 4/10
9s - loss: 0.2595 - acc: 0.9148 - val_loss: nan - val_acc: 0.9188
Epoch 5/10
9s - loss: 0.2285 - acc: 0.9235 - val_loss: nan - val_acc: 0.9225
Epoch 6/10
9s - loss: 0.2129 - acc: 0.9273 - val_loss: nan - val_acc: 0.9272
Epoch 7/10
9s - loss: 0.1995 - acc: 0.9309 - val_loss: nan - val_acc: 0.9284
Epoch 8/10
9s - loss: 0.1928 - acc: 0.9327 - val_loss: nan - val_acc: 0.9307
Epoch 9/10
9s - loss: 0.1881 - acc: 0.9336 - val_loss: nan - val_acc: 0.9322
Epoch 10/10
9s - loss: 0.1837 - acc: 0.9350 - val_loss: nan - val_acc: 0.9321
new jersey est parfois calme en l' automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


### Model 3: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [10]:
from keras.layers import Bidirectional


def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    rnn_size = english_vocab_size    #--- I found that setting the rnn output size to the vocab size of the input works 
    sentence_shape = (input_shape[1], input_shape[2])
    
    rnn_input  = Input(shape=sentence_shape)
    rnn        = Bidirectional(GRU(rnn_size, return_sequences=True)) (rnn_input)
    rnn        = TimeDistributed(Dense(french_vocab_size))           (rnn)
    rnn_output = Activation("softmax")                               (rnn)
    
    model = Model(inputs=rnn_input, outputs=rnn_output)
    
    learning_rate = 0.0075
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model
tests.test_bd_model(bd_model)


# Reshaping the input to work with bd_model (exactly like the simple_rnn model)
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[1], 1))

# Train the neural network
simple_bd_model = bd_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index),
    len(french_tokenizer.word_index))

simple_bd_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2, verbose=2)

# Print prediction(s)
print(logits_to_text(simple_bd_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
17s - loss: 1.5450 - acc: 0.6096 - val_loss: nan - val_acc: 0.6668
Epoch 2/10
16s - loss: 1.0269 - acc: 0.6832 - val_loss: nan - val_acc: 0.6921
Epoch 3/10
16s - loss: 0.9009 - acc: 0.7023 - val_loss: nan - val_acc: 0.7050
Epoch 4/10
16s - loss: 0.8341 - acc: 0.7138 - val_loss: nan - val_acc: 0.7206
Epoch 5/10
16s - loss: 0.7794 - acc: 0.7248 - val_loss: nan - val_acc: 0.7334
Epoch 6/10
16s - loss: 0.7392 - acc: 0.7349 - val_loss: nan - val_acc: 0.7260
Epoch 7/10
16s - loss: 0.7060 - acc: 0.7438 - val_loss: nan - val_acc: 0.7540
Epoch 8/10
16s - loss: 0.6804 - acc: 0.7554 - val_loss: nan - val_acc: 0.7447
Epoch 9/10
16s - loss: 0.6422 - acc: 0.7737 - val_loss: nan - val_acc: 0.7752
Epoch 10/10
16s - loss: 0.6403 - acc: 0.7653 - val_loss: nan - val_acc: 0.7670
new jersey est parfois calme au mois de et et neigeux chaud en en <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


### Model 4: Encoder-Decoder (OPTIONAL)
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

Create an encoder-decoder model in the cell below.

In [11]:
from keras.layers import RepeatVector


def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # OPTIONAL: Implement
    #--- thanks for the seq-to-seq tutorial from: 
    #--- https://blog.keras.io/building-autoencoders-in-keras.html
    #--- the code below uses the pattern in the seq-to-seq tutorial on that blog
    
    rnn_size = english_vocab_size    #--- I found that setting the rnn output size to the vocab size of the input works 
    
    sentence_shape = (input_shape[1], input_shape[2])
    
    encoder_rnn_input  = Input(shape=sentence_shape)
    encoder_rnn_output = GRU(rnn_size)                             (encoder_rnn_input) #--- dont'return a sequence!
    
    decoder_rnn_input  = RepeatVector(output_sequence_length)      (encoder_rnn_output)
    decoder_rnn        = GRU(rnn_size, return_sequences=True)      (decoder_rnn_input)
    decoder_rnn        = TimeDistributed(Dense(french_vocab_size)) (decoder_rnn)
    decoder_rnn_output = Activation("softmax")                     (decoder_rnn)
    
    model = Model(inputs=encoder_rnn_input, outputs=decoder_rnn_output)
    
    learning_rate = 0.0075
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model

tests.test_encdec_model(encdec_model)

# OPTIONAL: Train and Print prediction(s)

# Reshaping the input to work with a basic RNN (for use in encdec_model)
tmp_x = pad(preproc_english_sentences, preproc_english_sentences.shape[1]) #--- pad it to the length of the english sentence
tmp_x = tmp_x.reshape((-1, preproc_english_sentences.shape[1], 1))

# Train the neural network
simple_encdec_model = encdec_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],    #--- size of output will be length of the french translation
    len(english_tokenizer.word_index),
    len(french_tokenizer.word_index))

simple_encdec_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2, verbose=2)

# Print prediction(s)
print(logits_to_text(simple_encdec_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
15s - loss: 2.4621 - acc: 0.4669 - val_loss: nan - val_acc: 0.5442
Epoch 2/10
14s - loss: 1.6356 - acc: 0.5754 - val_loss: nan - val_acc: 0.6011
Epoch 3/10
14s - loss: 1.4219 - acc: 0.6062 - val_loss: nan - val_acc: 0.6262
Epoch 4/10
14s - loss: 1.3279 - acc: 0.6283 - val_loss: nan - val_acc: 0.6432
Epoch 5/10
14s - loss: 1.2958 - acc: 0.6345 - val_loss: nan - val_acc: 0.6538
Epoch 6/10
14s - loss: 1.1646 - acc: 0.6610 - val_loss: nan - val_acc: 0.6606
Epoch 7/10
14s - loss: 1.1724 - acc: 0.6565 - val_loss: nan - val_acc: 0.6687
Epoch 8/10
14s - loss: 1.1861 - acc: 0.6524 - val_loss: nan - val_acc: 0.6728
Epoch 9/10
14s - loss: 1.1656 - acc: 0.6566 - val_loss: nan - val_acc: 0.6581
Epoch 10/10
14s - loss: 1.0992 - acc: 0.6693 - val_loss: nan - val_acc: 0.6726
new jersey est parfois agrã©able en l' et il est il en en <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


### Model 5: Custom (IMPLEMENTATION)
Use everything you learned from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.

In [12]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    rnn_size = english_vocab_size    #--- I found that setting the rnn output size to the vocab size of the input works 
    
    embedding_input    = Input(shape=(input_shape[1],))
    embedding          = Embedding(english_vocab_size, 64, input_length=input_shape[1]) (embedding_input)
    encoder_rnn_output = Bidirectional(GRU(rnn_size))                                   (embedding)
    decoder_rnn_input  = RepeatVector(output_sequence_length)                           (encoder_rnn_output)
    decoder_rnn        = Bidirectional(GRU(rnn_size, return_sequences=True))            (decoder_rnn_input)
    decoder_rnn        = TimeDistributed(Dense(french_vocab_size))                      (decoder_rnn)
    decoder_rnn_output = Activation("softmax")                                          (decoder_rnn)
    
    model = Model(inputs=embedding_input, outputs=decoder_rnn_output)

    learning_rate = 0.0075
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model

tests.test_model_final(model_final)


print('Final Model Loaded')

Final Model Loaded


## Prediction (IMPLEMENTATION)

In [13]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences


def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # TODO: Train neural network using model_final
    padded_x = pad(x, x.shape[1])
    embeddable_x = padded_x.reshape((-1, x.shape[1]))
    model = model_final(
        embeddable_x.shape,
        y.shape[1],
        len(x_tk.word_index),
        len(y_tk.word_index))

    model.fit(embeddable_x, y, batch_size=1024, epochs=10, validation_split=0.2, verbose=2)
    
    #--- note to instructors: 
    #--- for Sample 2 (the second line) I think you meant to use np.max(x) instead of np.argmax(x)
    #--- i.e use this instead: print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))
    #--- reason: [argmax(x) for x in y[0]] returns [0 0 0 ...]
    
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.argmax(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
31s - loss: 2.1932 - acc: 0.5086 - val_loss: nan - val_acc: 0.6159
Epoch 2/10
29s - loss: 1.1728 - acc: 0.6761 - val_loss: nan - val_acc: 0.7230
Epoch 3/10
29s - loss: 0.8504 - acc: 0.7526 - val_loss: nan - val_acc: 0.7928
Epoch 4/10
29s - loss: 0.6166 - acc: 0.8156 - val_loss: nan - val_acc: 0.8612
Epoch 5/10
29s - loss: 0.3832 - acc: 0.8875 - val_loss: nan - val_acc: 0.9106
Epoch 6/10
29s - loss: 0.2411 - acc: 0.9311 - val_loss: nan - val_acc: 0.9125
Epoch 7/10
29s - loss: 0.1819 - acc: 0.9473 - val_loss: nan - val_acc: 0.9540
Epoch 8/10
29s - loss: 0.1353 - acc: 0.9607 - val_loss: nan - val_acc: 0.9613
Epoch 9/10
29s - loss: 0.1169 - acc: 0.9653 - val_loss: nan - val_acc: 0.9642
Epoch 10/10
29s - loss: 0.1037 - acc: 0.9693 - val_loss: nan - val_acc: 0.9638
Sample 1:
il a vu un vieux camion jaune <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Il a vu un vieux camion jaune
Sample 2:
new 

## Submission
When you are ready to submit your project, do the following steps:
1. Ensure you pass all points on the [rubric](https://review.udacity.com/#!/rubrics/1004/view).
2. Submit the following in a zip file.
  - `helper.py`
  - `machine_translation.ipynb`
  - `machine_translation.html`
    - You can export the notebook by navigating to **File -> Download as -> HTML (.html)**.